In [7]:
DEPTH = 4

PATH_TO_CORPUS = "C:\\Users\\PLour\\OneDrive - Universidade Federal de Minas Gerais\\01_Estudos\\Faculdade\\RI\\Crawler\\Corpus"

# external modules
import sys 


# internal modules
import utils as ut


In [8]:
def sleep_delay(delay=None):
    if not delay:
        time.sleep(0.1)
    else:
        time.sleep(delay)

In [9]:
# 3
import requests
from bs4 import BeautifulSoup
url = 'https://exame.com/'
page = requests.get(url, auth=('user', 'pass'))
soup = BeautifulSoup(page.content, 'html.parser')



In [10]:
import time
import utils as ut
url_queue = {}
url_queue_ = {}
url_queue_ = ut.add_url_to_queue(soup, url, url_queue_, _url_depth=DEPTH)
url_recusado = {}
_previous_url = ''
allowed, delay, robots= True, None, None
request_time_list = []
robots_time_list = []

for i, url in enumerate(url_queue_):
    
    if i > 100:
        break

    _url_prefix = url.split('/', maxsplit=1)[0] 
    _previous_url_prefix = _previous_url.split('/', maxsplit=1)[0]
    
    domain, url = ut.process_url(url)
    
    lap_time = time.time()
    
    if _url_prefix != _previous_url_prefix:
        # print("awake")
        allowed, robots = ut.check_robots(domain)
    else:
        allowed, delay = ut.check_robots(domain, robots)
        # print("sleep")
        sleep_delay(delay)
        
    robots_time_list.append(time.time() - lap_time)

    if not allowed:
        print('not allowed in: ' + url)
        continue

    state = ['NADA', 'PAGE', 'SOUP', 'QUEUE', 'OVER']
    iter_state = 0
    try:
        iter_state = 0
        lap_time = time.time()
        page = requests.get(url, auth=('user', 'pass'))
        request_time_list.append(time.time() - lap_time)
        iter_state = 1

        soup = BeautifulSoup(page.content, 'html.parser')
        iter_state = 2
        old_len = len(url_queue)
    except: 
        url_recusado[url] = url
        print("")
        print("[STATE: " + state[iter_state] + "] couldn't access: " + url)
        request_time_list.append(time.time() - lap_time)

    url_queue = ut.add_url_to_queue(soup, url, url_queue, _url_depth=DEPTH)
    _previous_url = url[7:]

    

not allowed in: http://exame-faq.force.com/s


In [15]:
import numpy as np

np_request_time_list = np.array(request_time_list)
np_robots_time_list = np.array(robots_time_list)
np_request_time_list.mean(), np_robots_time_list.mean()
#np_request_time_list.shape, np_robots_time_list.shape


(2.1901253509521483, 0.30202245240164277)

In [14]:
len(url_queue), len(url_queue_), len(url_recusado)

(38, 207, 0)